# 3D example showcasing some of the features of SMART

Geometry is divided into 4 domains; two volumes, and two surfaces:
- cytosol (Cyto): $\Omega_{Cyto}$
- endoplasmic reticulum volume (ER): $\Omega_{ER}$
- plasma membrane (PM): $\Gamma_{PM}$
- ER membrane (ERm): $\Gamma_{ERm}$

For simplicity, here we consider a "cube-within-a-cube" geometry, in which the smaller
inner cube represents a section of ER and one face of the outer cube (x=0) represents the PM. The other
faces of the outer cube are treated as no flux boundaries. The space outside
the inner cube but inside the outer cube is classified as cytosol.

There are three function-spaces on these three domains:
$$
u^{Cyto} = [A, B] \quad \text{on} \quad \Omega^{Cyto}\\
u^{ER} = [AER] \quad \text{on} \quad \Omega^{ER}\\
v^{ERm} = [R, Ro] \quad \text{on} \quad \Gamma^{ERm}
$$

In words, this says that species A and B reside in the cytosolic volume, 
species AER corresponds to an amount of species A that lives in the ER volume,
and species R (closed receptor/channel) and Ro (open receptor/channel) reside on the ER membrane.

In this model, species B reacts with a receptor/channel, R, on the ER membrane, causing it to open (change state from R->Ro), 
allowing species A to flow out of the ER and into the cytosol. 
Note that this is roughly similar to an IP3 pulse at the PM, leading to Ca2+ release from the ER,
where, by analogy, species B is similar to IP3 and species A is similar to Ca2+. A more comprehensive
model of Ca2+ dynamics in particular is implemented in Example 4.


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
img_A = mpimg.imread('example5-diagram.png')
plt.imshow(img_A)
plt.axis('off')

As specified in our mathematical documentation, assuming diffusive transport, the PDE and boundary condition for each of these volumetric species takes the form:
$$
\partial_t u_i^m - \nabla \cdot ( D_i^m \nabla (u_i^m) ) - f_i^m(u_i^m) = 0 \qquad \text{ in } \Omega^m\\
D_i \nabla u_i^m \cdot n^m - R_i^{q} (u^m, u^n, v^q) = 0 \qquad \text{ on } \Gamma^{q}
$$

and the surface species take the form:
$$
\partial_t v_i^q - \nabla_S \cdot (D_i^q \nabla_S v ) - g_i^q ( u^m, u^n, v^q ) = 0 \qquad \text{ on } \Gamma^{q}\\
D_i \nabla v_i^q \cdot n^q = 0 \qquad \text{ on } \partial\Gamma^{q}
$$

Our reaction terms and boundary conditions are chosen according to the system described above. For the purposes of this simplified example we use linear mass action in all reaction terms. Explicitly writing out this system of PDEs, we have:

\begin{align}
    \partial_t u_B^{Cyto} - D_B^{Cyto} \nabla^2 u_B^{Cyto} + k_{2f} u_B^{Cyto} &= 0 \qquad \text{ in } \Omega^{Cyto}\\
    D_B^{Cyto} \nabla u_B^{Cyto} \cdot n^{Cyto} + j_1[t] &= 0 \qquad \text{ on } \Gamma^{PM} \nonumber\\
    D_B^{Cyto} \nabla u_B^{Cyto} \cdot n^{Cyto} + k_{3f} v_R^{ERm} u_B^{Cyto} - k_{3r} v_{Ro}^{ERm} &= 0 \qquad \text{ on } \Gamma^{ERm} \nonumber\\
    \nonumber \\
    \partial_t u_A^{Cyto} - D_A^{Cyto} \nabla^2 u_A^{Cyto} &= 0 \qquad \text{ in } \Omega^{Cyto}\\
    D_A^{Cyto} \nabla u_A^{Cyto} \cdot n^{Cyto} &= 0 \qquad \text{ on } \Gamma^{PM} \nonumber\\
    D_A^{Cyto} \nabla u_A^{Cyto} \cdot n^{Cyto} - k_{4,Vmax} v_{Ro}^{ERm} (u_{AER}^{ER} - u_A^{Cyto}) &= 0 \qquad \text{ on } \Gamma^{ERm} \nonumber\\
    \nonumber \\
    \partial_t u_{AER}^{ER} - D_{AER}^{ER} \nabla^2 u_{AER}^{ER} &= 0 \qquad \text{ in } \Omega^{ER}\\
    D_{AER}^{ER} \nabla u_{AER}^{ER} \cdot n^{ER} + k_{4,Vmax} v_{Ro}^{ERm} (u_{AER}^{ER} - u_A^{Cyto}) &= 0 \qquad \text{ on } \Gamma^{ERm} \nonumber\\
    \nonumber \\
    \partial_t v_{R}^{ERm} - D_{R}^{ERm} \nabla^2 v_{R}^{ERm} - 
    k_{3f} v_R^{ERm} u_B^{Cyto} + k_{3r} v_{Ro}^{ERm}  &= 0 \qquad \text{ on } \Gamma^{ERm}\\
    \nonumber \\
    \partial_t v_{Ro}^{ERm} - D_{Ro}^{ERm} \nabla^2 v_{Ro}^{ERm} +
    k_{3f} v_R^{ERm} u_B^{Cyto} - k_{3r} v_{Ro}^{ERm} &= 0 \qquad \text{ on } \Gamma^{ERm}\\
\end{align}

## Code imports and initialization

In [ ]:
import os
import logging

import dolfin as d
import sympy as sym
import numpy as np

from smart import config, common, mesh, model, mesh_tools
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from smart.units import unit

We will set the logging level to `INFO`. This will display some output during the simulation. If you want to get even more output you could set the logging level to `DEBUG`.

In [ ]:
logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

Futhermore, you could also save the logs to a file by attaching a file handler to the logger as follows.

```python
file_handler = logging.FileHandler("filename.log")
file_handler.setFormatter(logging.Formatter(smart.config.base_format))
logger.addHandler(file_handler)
```

First, we define the various units for the inputs

In [ ]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM
surf_unit = molecule / um**2

## Generate model
Next we generate the model described in the equations above.

### Compartments
As described above, the three compartments are the cytosol ("Cyto"), the plasma membrane ("PM"), the ER membrane ("ERm"), and the ER interior volume ("ER"). These are initialized by calling:
```python
compartment_name = Compartment(name, dim, units, marker)
```
where
- name = string naming the compartment
- dim = topological dimensionality (e.g. 3 for Cyto, 2 for PM)
- units = length units for the compartment (um for all here)
- marker = marker value identifying each compartment in the parent mesh

Note that, as shown, we can also specify nonadjacency for compartments; this is not strictly necessary, but will generally speed up the simulations.

In [ ]:
Cyto = Compartment("Cyto", 3, um, 1)
PM = Compartment("PM", 2, um, 10)
ER = Compartment("ER", 3, um, 2)
ERm = Compartment("ERm", 2, um, 12)
PM.specify_nonadjacency(['ERm', 'ER'])
ERm.specify_nonadjacency(['PM'])

Initialize a compartment container and add the 4 compartments to it.

In [ ]:
cc = CompartmentContainer()
cc.add([ERm, ER, PM, Cyto])

### Species
In this case, we have 5 species across 3 different compartments. Each is initialized by calling:
```python
species_name = Species(name, conc_init, conc_units, D, D_units, compartment)
```
where
- name = string naming the species
- conc_init = initial concentration for this species (as in the case of Rinit here, can be an expression given by a string to be parsed by sympy - the only unknowns in the expression should be x, y, and z)
- conc_units = concentration units for this species (micromolar/uM for volumetric species here and molecules/um**2 for surface species)
- D = value of diffusion coefficient
- D_units = units for diffusion coefficient (um**2/sec here)
- compartment = each species should be assigned to a single compartment (either "Cyto", "ER" or "ERm" here)

In [ ]:
A = Species("A", 0.01, vol_unit, 1.0, D_unit, "Cyto")
B = Species("B", 0.0, vol_unit, 1.0, D_unit, "Cyto")
AER = Species("AER", 200.0, vol_unit, 5.0, D_unit, "ER")
#Create an algebraic expression to define the initial condition of R
Rinit = "(sin(40*y) + cos(40*z) + sin(40*x) + 3) * (y-x)**2"
R1 = Species("R1", Rinit, surf_unit, 0.02, D_unit, "ERm")
R1o = Species("R1o", 0.0, surf_unit, 0.02, D_unit, "ERm")

Create species container and add the 5 species objects to it.

In [ ]:
sc = SpeciesContainer()
sc.add([R1o, R1, AER, B, A])

###  Parameters and Reactions

Parameters and reactions are generally defined together, although the order does not strictly matter. Parameters are specified as:
```python
param_name = Parameter(name, value, units, group (opt))
```
where
- name: string naming the parameter
- value: value of the given parameter
- units: units associated with given value
- group: optional string placing this reaction in a reaction group; for organizational purposes when there are multiple reaction modules

Reactions are specified by a variable number of arguments (optional arguments are indicated by (opt) here):
```python
reaction_name = Reaction(name, reactants, products, param_map
                         eqn_f_str (opt), eqn_r_str (opt), reaction_type (opt), species_map (opt), 
                         explicit_restriction_to_domain (opt), group (opt), flux_scaling (opt))
```
required arguments:
- name: string naming the reaction
- reactants: list of strings specifying the reactants for this reaction
- products: list of strings specifying the products for this reaction
    ***NOTE: the lists "reactants" and "products" determine the stoichiometry of the reaction,
       for instance, if two A's react to give one B, the reactants list would be ["A","A"],
       and the products list would be ["B"]
- param_map: relationship between the parameters specified in the reaction string and those given
              in the parameter container. By default, the reaction parameters are "on" and "off" when
              a system obeys simple mass action. If the forward rate is given by a parameter "k1" and the
              reverse rate is given by "k2", then param_map = {"on":"k1", "off":"k2"}

optional arguments:
- eqn_f_str: For systems not obeying simple mass action, this string specifies the forward reaction rate
             By default, this string is "on*{all reactants multiplied together}"
- eqn_r_str: For systems not obeying simple mass action, this string specifies the reverse reaction rate
             By default, this string is "off*{all products multiplied together}"
- reaction_type: either "custom" or "mass_action" (default is "mass_action") [never a required argument]
- species_map: same format as param_map; only required if the species name in the reaction string do not
               match the species names given in the species container
- explicit_restriction_to_domain: string specifying where the reaction occurs; required if the reaction is not
                                  constrained by the reaction string (e.g., if production occurs only at the boundary,
                                  as it does here, but the species being produced exists through the entire volume)
- group: string placing this reaction in a reaction group; for organizational purposes when there are multiple reaction modules
- flux_scaling: in certain cases, a given reactant or product may experience a scaled flux (for instance, if we assume that
                some of the molecules are immediately sequestered after the reaction); in this case, to signify that this flux 
                should be rescaled, we specify ''flux_scaling = {scaled_species: scale_factor}'', where scaled_species is a
                string specifying the species to be scaled and scale_factor is a number specifying the rescaling factor

In [ ]:
# Degradation of B in the cytosol
k2f = Parameter("k2f", 10, 1 / sec)
r2 = Reaction(
    "r2", ["B"], [], param_map={"on": "k2f"}, reaction_type="mass_action_forward"
)

# Activating receptors on ERm with B
k3f = Parameter("k3f", 100, 1 / (uM * sec))
k3r = Parameter("k3r", 100, 1 / sec)
r3 = Reaction("r3", ["B", "R1"], ["R1o"], {"on": "k3f", "off": "k3r"})
# Release of A from ERm to cytosol
k4Vmax = Parameter("k4Vmax", 2000, 1 / (uM * sec))
r4 = Reaction(
    "r4",
    ["AER"],
    ["A"],
    param_map={"Vmax": "k4Vmax"},
    species_map={"R1o": "R1o", "uER": "AER", "u": "A"},
    eqn_f_str="Vmax*R1o*(uER-u)",
)

We define one additional reaction as the time-dependent production of species B at the plasma membrane. In this case, we define a pulse-type function as the derivative of an arctan function. Note that this is useful because we can provide an expression to use in pre-integration.
$$
j_{int}[t] = V_{max} \arctan\left({m (t - t_0)}\right)\\
j_1[t] = \int{j_{int}[t]}dt = \frac{m V_{max}}{1 + m^2 (t-t_0)^2}
$$

In [ ]:
Vmax, t0, m = 500, 0.1, 200
t = sym.symbols("t")
pulseI = Vmax * sym.atan(m * (t - t0))
pulse = sym.diff(pulseI, t)
j1pulse = Parameter.from_expression(
    "j1pulse", pulse, flux_unit, use_preintegration=True, preint_sym_expr=pulseI
)
r1 = Reaction(
    "r1",
    [],
    ["B"],
    param_map={"J": "j1pulse"},
    eqn_f_str="J",
    explicit_restriction_to_domain="PM",
)

We can plot the time-dependent input by converting the sympy expression to a numpy function using lambdify (this is how SMART evaluates these expressions internally).

In [ ]:
from sympy.utilities.lambdify import lambdify
pulse_func = lambdify(t, pulse,'numpy') # returns a numpy-ready function
tArray = np.linspace(0, 1, 100)
pulse_vals = pulse_func(tArray)
plt.plot(tArray, pulse_vals)

Note that we can also create a time-dependent parameter by reading data from a csv file. This is illustrated below, introducing a new parameter, "j1pulse_fromfile", which we do not use in the full model in this case. It could be used in the model by simply defining it as "j1pulse" to overwrite the parameter previously defined from an expression.

In [ ]:
j1pulse_fromfile = Parameter.from_file("j1pulse", "sample_input.csv", flux_unit)
tArray = j1pulse_fromfile.sampling_data[:,0]
pulse_vals = j1pulse_fromfile.sampling_data[:,1]
plt.plot(tArray, pulse_vals)

Create containers for parameters and reactions and add all the parameters and reaction objects to them.

In [ ]:
pc = ParameterContainer()
pc.add([k4Vmax, k3r, k3f, k2f, j1pulse])

rc = ReactionContainer()
rc.add([r1, r2, r3, r4])

## Create/load in mesh

In SMART we have different levels of meshes:
- Parent mesh: contains the entire geometry of the problem, including all surfaces and volumes
- Child meshes: submeshes (sections of the parent mesh) associated with individual compartments. Here, the child meshes are:
    - Cyto: the portion of the outer cube outside of the inner cube, defined by `cell_markers = 1`
    - ER: the inside portion of the inner cube, defined by `cell_markers = 2`
    - PM: surface mesh where x=0, defined by `facet_markers = 10`
    - ERm: surface mesh corresponding to all faces of the inner cube, defined by `facet_markers = 12`

Here we create a UnitCube mesh as the Parent mesh, defined by

$$
\Omega = [0, 1] \times [0, 1] \times [0, 1] \subset \mathbb{R}^3
$$

In [ ]:
domain, facet_markers, cell_markers = mesh_tools.DemoCuboidsMesh()

By default, DemoCuboidsMesh marks all faces of the outer cube as "10", our marker value associated with PM. Here, since we are only treating the x=0 face as PM, we alter the facet markers on all other faces, setting them equal to zero. They are then treated as no-flux boundaries not belonging to a designated surface compartment. The resultant mesh with the new facet and volume markers is displayed below.

In [ ]:
# Turn off "PM" on all sides of the cube except x=0
for face in d.faces(domain):
    if face.midpoint().x() > d.DOLFIN_EPS and facet_markers[face] == 10:
        facet_markers[face] = 0
img_mesh = mpimg.imread('example5-mesh.png')
plt.imshow(img_mesh)
plt.axis('off')

We now save the mesh as an h5 file and then read it into SMART as a `ParentMesh` object. 

In [ ]:
os.makedirs("mesh", exist_ok=True)
mesh_tools.write_mesh(domain, facet_markers, cell_markers, filename="mesh/DemoCuboidsMesh")
parent_mesh = mesh.ParentMesh(
    mesh_filename="mesh/DemoCuboidsMesh.h5",
    mesh_filetype="hdf5",
    name="parent_mesh",
)

## Model and solver initialization

Now we are ready to set up the model. First we load the default configurations and set some configurations for the current solver.

In [ ]:
conf = config.Config()
conf.solver.update(
    {
        "final_t": 1,
        "initial_dt": 0.01,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)
print(conf)

We create a model using the different containers and the parent mesh. For later reference, we save the model information as a pickle file. 

In [ ]:
model_cur = model.Model(pc, sc, cc, rc, conf, parent_mesh)
model_cur.to_pickle('model_cur.pkl')
print(model_cur)

Note that we could later load the model information from the pickle file using the line:
```Python
model_cur = model.from_pickle(model_cur.pkl)
```

Next we need to initialize the model and solver.

In [ ]:
model_cur.initialize(initialize_solver=False)
model_cur.initialize_discrete_variational_problem_and_solver()

## Solving system and storing data

We create some XDMF files where we will store the output 

In [ ]:

# Write initial condition(s) to file
results = dict()
os.makedirs("results", exist_ok=True)
for species_name, species in model_cur.sc.items:
    results[species_name] = d.XDMFFile(
        model_cur.mpi_comm_world, f"results/{species_name}.xdmf"
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)

We now run the the solver and store the data at each time point to the initialized files. We also integrate A over the cytosolic volume at each time step to monitor the elevation in A over time.

In [ ]:
avg_A = []
tvec = []
# Solve
while model_cur.t <  model_cur.final_t:
    # Solve the system
    model_cur.monolithic_solve()
    # Save results for post processing
    for species_name, species in model_cur.sc.items:
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
        # compute average Aphos concentration at each time step
    dx = d.Measure("dx",domain = model_cur.cc['Cyto'].dolfin_mesh)
    int_val = d.assemble(model_cur.sc['A'].u['u']*dx)
    volume = d.assemble(1.0*dx)
    avg_A.append(int_val / volume)
    tvec.append(model_cur.t)

Finally, we plot the average concentration of A over time.

In [ ]:
plt.plot(tvec, avg_A)
plt.xlabel('Time (s)')
plt.ylabel('Cytosolic concentration of A (μM)')